<a href="https://colab.research.google.com/github/young-hwanlee/my-practical-statistics-for-data-scientists/blob/main/Chapter_3_Statistical_Experiments_and_Significance_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Practical Statistics for Data Scientists (Python)**
# **Chapter 3 - Statistical Experiments and Significance Testing**
> (c) 2019 Peter C. Bruce, Andrew Bruce, and Peter Gedeck

Import required Python packages.

In [ ]:
% 

Define paths to data sets. If you don't keep your data in the same directory as the code, adapt the path names.

## **3.3 Resampling**

The following code is different to the R version. idx_A and idx_B are reversed.

## **3.4 Statistical Significance and P-Value**
### **3.4.1 P-Value**



If np.mean is applied to a list of booleans, it gives the percentage of how often True was found in the list (#True / #Total).

## **3.5 t-Tests**

## **3.8 ANOVA (ANalysis Of VAriance)**

### **3.8.1 F-Statistic**

We can compute an ANOVA table using *statsmodels*.

### **3.8.2 Two-Way ANOVA**
### **Two-way ANOVA is only available with statsmodels**  
> formula = 'len ~ C(supp) + C(dose) + C(supp):C(dose)'  
> model = ols(formula, data).fit()  
> aov_table = anova_lm(model, typ=2)

## **3.9 Chi-Square Test**
### **3.9.1 Chi-Square Test: A Resampling Approach**

### **3.9.2 Figure Chi-Square Distribution**

### **3.9.3 Fisher's Exact Test**

*Scipy* has only an implementation of Fisher's Exact Test for 2x2 matrices. There is a github repository that provides a Python implementation that uses the same code as the R version. Installing this requires a Fortran compiler.  
> stat.fisher_exact(clicks)

### **Scientific Fraud**

## **3.11 Power and Sample Size**

*Statsmodels* has a number of methods for power calculation.  
See e.g.: https://machinelearningmastery.com/statistical-power-and-power-analysis-in-python/